In [1]:
# import packages
import pickle
from flair.models import SequenceTagger
from flair.models import RelationExtractor
from flair.data import Sentence, Corpus
import os
import logging
import pandas as pd
from flair.data import Sentence, Corpus

In [2]:
initial_model = 'models/final-model.pt'
ner_data_dir = 'PATTERNS/CORPUS_GROUPS(NER)/'
re_data_dir = 'PATTERNS/CORPUS_GROUPS/'
ERPATTERN_FILE = 'PATTERNS/CASE_LAW_COUNSEL/cl_counsel_re_patterns_DO_NOT_DELETE_4_annotation_Afsar_v4.xlsx'

Here we found we cannot load 'final-model.pt' with any version of pytorch. So we try some different models instead.
* 'ner-fast' NER
* 'relations-fast' RE

In [3]:
Sentence_TEXT = pd.read_excel(ERPATTERN_FILE, sheet_name='ER_PATTERNS')
sents = []
loaded_model_ner = SequenceTagger.load('ner-fast')
loaded_model_re = RelationExtractor.load('relations-fast')
# loaded_model_ner = SequenceTagger.load(initial_model)
# loaded_model_re = RelationExtractor.load(initial_model)


2021-12-05 18:20:02,718 --------------------------------------------------------------------------------
2021-12-05 18:20:02,718 The model key 'ner-fast' now maps to 'https://huggingface.co/flair/ner-english-fast' on the HuggingFace ModelHub
2021-12-05 18:20:02,718  - The most current version of the model is automatically downloaded from there.
2021-12-05 18:20:02,719  - (you can alternatively manually download the original model at https://nlp.informatik.hu-berlin.de/resources/models/ner-fast/en-ner-fast-conll03-v0.4.pt)
2021-12-05 18:20:02,719 --------------------------------------------------------------------------------
2021-12-05 18:20:04,875 loading file /home/hdu5/.flair/models/ner-english-fast/4c58e7191ff952c030b82db25b3694b58800b0e722ff15427f527e1631ed6142.e13c7c4664ffe2bbfa8f1f5375bd0dced866b8c1dd7ff89a6d705518abf0a611
2021-12-05 18:20:05,932 loading file /home/hdu5/.flair/models/relations-fast.pt


In [4]:
for index, row in Sentence_TEXT.iterrows():
    sents.append(row['Sentence_TEXT'])
print(len(sents))
# print(sents)
dict_of_entity = {}
index = 1
ke_list = []

718


In [5]:
sents

['Attorney General Joshua H . Stein , by Special Deputy Attorney General Marc X . Sneed , for the State .',
 'Attorney General Joshua H . Stein , by Special Deputy Attorney General Marc X . Sneed , for the State .',
 'Remington S . Dalke , county attorney , and Derek Schmidt , attorney general , for appellant .',
 'Remington S . Dalke , county attorney , and Derek Schmidt , attorney general , for appellant .',
 'Lori Siegel , under appointment by the Court of Appeal , for Minors and Appellants Ender H . and Jacob H .',
 'Amanda Furman , Ernest H . Nomura , Gabriele V . Provenza , Deputies Corporation Counsel , City and County of Honolulu , for Respondents - Appellees - Appellees .',
 'Amanda Furman , Ernest H . Nomura , Gabriele V . Provenza , Deputies Corporation Counsel , City and County of Honolulu , for Respondents - Appellees - Appellees .',
 'Amanda Furman , Ernest H . Nomura , Gabriele V . Provenza , Deputies Corporation Counsel , City and County of Honolulu , for Respondents - 

In [6]:
sentence = Sentence(sents[0])
print(sentence)

Sentence: "Attorney General Joshua H . Stein , by Special Deputy Attorney General Marc X . Sneed , for the State ."   [− Tokens: 21]


The following shows an example extracted from the original data file.

In [10]:
loaded_model_ner.predict(sentence)

In [11]:
entities = sentence.get_labels('ner')

In [12]:
entities

[PER [Joshua H (3,4)] (0.8591),
 PER [Stein (6)] (0.9762),
 PER [Marc X (13,14)] (0.9096),
 PER [Sneed (16)] (0.8113),
 ORG [State (20)] (0.392)]

Here we try to apply the previous example relation extraction to all the data.

In [14]:
for i in range(len(sents)):
    if i > 500:
        break
    print(str(i) + ' ' + '-' * 20)
    sentence = Sentence(sents[i])
    loaded_model_ner.predict(sentence)
    loaded_model_re.predict(sentence)
    relations = sentence.get_labels('relation')
    for relation in relations:
        head = relation.head.text
        tail = relation.tail.text
        print(head)
        print(tail)
        # input()
        if dict_of_entity.get(head) is None:
            dict_of_entity[head] = index
            head_int = index
            index += 1
        else:
            head_int = dict_of_entity[head]

        if dict_of_entity.get(tail) is None:
            dict_of_entity[tail] = index
            tail_int = index
            index += 1
        else:
            tail_int = dict_of_entity[tail]

        ke_list.append([head_int, tail_int])

with open('entity_dictionary.pkl', 'wb') as f:
    pickle.dump(dict_of_entity, f)

# with open('saved_dictionary.pkl', 'rb') as f:
#     load_entity_dict = pickle.load(f)

with open('relation.pkl', 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(ke_list, filehandle)

0 --------------------
Joshua H
State
Marc X
State
1 --------------------
Joshua H
State
Marc X
State
2 --------------------
3 --------------------
4 --------------------
5 --------------------
6 --------------------
7 --------------------
8 --------------------
9 --------------------
10 --------------------
11 --------------------
12 --------------------
13 --------------------
14 --------------------
Claire Pavlovic
Public Defender Service
15 --------------------
Claire Pavlovic
Public Defender Service
16 --------------------
Claire Pavlovic
Public Defender Service
17 --------------------
Jeffrey Bossert Clark
Commercial Litigation Branch
Jeanne E
Commercial Litigation Branch
18 --------------------
Jeffrey Bossert Clark
Commercial Litigation Branch
Jeanne E
Commercial Litigation Branch
19 --------------------
Jeffrey Bossert Clark
Commercial Litigation Branch
Jeanne E
Commercial Litigation Branch
20 --------------------
Jeffrey Bossert Clark
Commercial Litigation Branch
Jeanne E
Com

At here we have extracted the relations from the orignal file (the xlsx file).